## Import

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import mplhep as hep
hep.style.use("CMS")

## Load file and specify the board name

In [ ]:
sql_file = '/media/daq/T7/ETROC-History-DESY/BaselineHistory.sqlite'
con = sqlite3.connect(sql_file)
df = pd.read_sql_query("SELECT * from baselines", con)

## Print the board name

In [ ]:
column_values = df['chip_name']

# Print the unique elements in the column
unique_elements = column_values.unique()
for element in unique_elements:
    print(element)

## Print the saved notes for dedicate board name

In [ ]:
chip_name = 'ET2p01_Bar4'
column_values = df[(df['chip_name'] == chip_name)]['note']

# Print the unique elements in the column
unique_elements = column_values.unique()
for element in unique_elements:
    if('22C' in element):
        print(element)

## Select the data based on the board name and note

In [ ]:
col = 8
row = 8

In [ ]:
base_df = df[(df['chip_name'] == chip_name) & (df['col'] == col) & (df['row'] == row)]
base_df

In [ ]:
def makePlot(var, inputname, scan_pixels, timestamp_cut=None, note_cut=None, label=None):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(24,9))
    ax.set_prop_cycle(color=['#e41a1c','#fdbf6f','#d95f02', '#377eb8','#4daf4a','#b2df8a',])

    i = 0

    threshold_timestamp = pd.Timestamp(timestamp_cut)
    for row, col in scan_pixels:
        filtered_df = df[(df['chip_name'] == inputname) & (df['row'] == row) & (df['col'] == col)  & (df['baseline'] != 0)]
        # filtered_df = df[(df['chip_name'] == inputname) & df.loc[row_indexer,col_indexer]  & (df['baseline'] != 0)]
        filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'])

        if note_cut is not None:
            filtered_df = filtered_df[filtered_df['note'].str.contains(note_cut)]
        if timestamp_cut is not None:
            filtered_df = filtered_df.loc[filtered_df['timestamp'] > threshold_timestamp]

        hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
        if label is not None:
            ax.set_title(label, size=17, loc="right")
        ax.plot(filtered_df['timestamp'], filtered_df[var], '.-', label=f'Row-Col = ({row}-{col})')

        # Set x-axis labels with line breaks using LaTeX notation
        if i == 0:
            date_form = DateFormatter("%Y-%m-%d %H:%M")
            ax.xaxis.set_major_formatter(date_form)
            plt.xticks(rotation=60)
            # x_labels = [f'{row["note"].replace(f"_{chip_name}","")}\n{row["timestamp"].strftime("%Y-%m-%d %H:%M")}' for index, row in filtered_df.iterrows()]
            # ax.set_xticks(filtered_df['timestamp'])
            # ax.set_xticklabels(x_labels, rotation=90, fontsize=9)

        i += 1

    ax.grid(which='both', axis='y')
    ax.xaxis.set_minor_locator(plt.NullLocator())
    ax.legend(ncols=2, fontsize=14)

    var_label = str(var).replace('_', ' ')

    # Set labels for other axes as needed
    ax.set_ylabel(f'{inputname} {var_label}')

In [ ]:
scan_list = [(row,col)] # (row, col)

time_cut = None # '2024-01-17 11:57:00'
note_cut = 'Overnight22C_Jan_18_2024_time_3s' # 'OvernightWarmup_Jan_18_2024_time_3s'
label = 'Overnight warmup from 0 to 22 C'

makePlot('baseline', chip_name, scan_list, time_cut, note_cut, label)
makePlot('noise_width', chip_name, scan_list, time_cut, note_cut, label)